In [ ]:
# 导入必要的库
import torch  # PyTorch 深度学习框架的核心库
from torch import nn  # 导入神经网络模块，用于构建网络层
from d2l import torch as d2l  # 导入《动手学深度学习》的工具库，提供训练和数据加载函数

In [ ]:
def conv_block(input_channels, num_channels):
    """
    卷积块函数：DenseNet的基本构建单元
    参数:
        input_channels: 输入通道数
        num_channels: 输出通道数
    返回:
        一个包含批量归一化、激活函数和卷积层的序列模块
    """
    return nn.Sequential(
        nn.BatchNorm2d(input_channels),  # 批量归一化层，对输入进行标准化，加速训练并提高稳定性
        nn.ReLU(),  # ReLU激活函数，引入非线性，将负值变为0
        nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1))  # 3x3卷积层，padding=1保持特征图尺寸不变

In [ ]:
class DenseBlock(nn.Module):
    """
    稠密块（Dense Block）：DenseNet的核心组件
    每一层的输入都是前面所有层输出的拼接，实现特征重用
    """
    def __init__(self, num_convs, input_channels, num_channels):
        """
        初始化稠密块
        参数:
            num_convs: 该稠密块中卷积层的数量
            input_channels: 输入通道数
            num_channels: 增长率，每个卷积块新增的通道数
        """
        super(DenseBlock, self).__init__()  # 调用父类的初始化方法
        layer = []  # 创建空列表，用于存储所有卷积块
        for i in range(num_convs):  # 循环创建num_convs个卷积块
            # 计算当前卷积块的输入通道数：原始输入 + 前面i个块的输出
            # 每个块输出num_channels个通道，所以前i个块共输出 i * num_channels 个通道
            layer.append(conv_block(
                num_channels * i + input_channels, num_channels))
        self.net = nn.Sequential(*layer)  # 将所有卷积块组合成一个序列模块

    def forward(self, X):
        """
        前向传播函数
        参数:
            X: 输入张量
        返回:
            拼接了所有层输出的张量
        """
        for blk in self.net:  # 遍历稠密块中的每个卷积块
            Y = blk(X)  # 将当前输入X通过卷积块得到输出Y
            # 在通道维度（dim=1）上将输入X和输出Y拼接
            # 这是DenseNet的关键：每层的输入包含前面所有层的特征
            X = torch.cat((X, Y), dim=1)
        return X  # 返回最终拼接的特征图

In [ ]:
# 测试稠密块的输出形状
blk = DenseBlock(2, 3, 10)  # 创建一个稠密块：2个卷积层，3个输入通道，增长率为10
X = torch.randn(4, 3, 8, 8)  # 创建随机输入张量：批量大小4，3个通道，8x8的特征图
Y = blk(X)  # 通过稠密块进行前向传播
# 输出形状应该是 (4, 3+2*10, 8, 8) = (4, 23, 8, 8)
# 因为输入3个通道 + 第1层输出10个通道 + 第2层输出10个通道 = 23个通道
print(Y.shape)  # 打印输出张量的形状

torch.Size([4, 23, 8, 8])


In [ ]:
def transition_block(input_channels, num_channels):
    """
    过渡层（Transition Layer）：连接两个稠密块
    作用：减少通道数和特征图尺寸，控制模型复杂度
    参数:
        input_channels: 输入通道数
        num_channels: 输出通道数（通常是输入通道数的一半）
    返回:
        包含批量归一化、激活、1x1卷积和平均池化的序列模块
    """
    return nn.Sequential(
        nn.BatchNorm2d(input_channels),  # 批量归一化
        nn.ReLU(),  # ReLU激活函数
        nn.Conv2d(input_channels, num_channels, kernel_size=1),  # 1x1卷积降低通道数
        nn.AvgPool2d(kernel_size=2, stride=2))  # 2x2平均池化，将特征图尺寸减半

In [ ]:
# 测试过渡层的输出形状
blk = transition_block(23, 10)  # 创建过渡层：23个输入通道，10个输出通道
# Y的形状是 (4, 23, 8, 8)，经过过渡层后：
# 1x1卷积将通道数从23降到10
# 2x2平均池化将特征图从8x8降到4x4
# 最终输出形状应该是 (4, 10, 4, 4)
print(blk(Y).shape)  # 打印输出张量的形状

torch.Size([4, 10, 4, 4])


In [ ]:
# 构建DenseNet的第一个模块（类似于ResNet的stem）
b1 = nn.Sequential(
    # 7x7大卷积核，步幅为2，padding为3，将1通道(灰度图)转换为64通道
    # 特征图尺寸减半 (96x96 -> 48x48)
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64),  # 对64个通道进行批量归一化
    nn.ReLU(),  # ReLU激活函数
    # 3x3最大池化，步幅为2，padding为1
    # 特征图尺寸再次减半 (48x48 -> 24x24)
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [ ]:
# 构建DenseNet的主体部分：多个稠密块和过渡层
num_channels, growth_rate = 64, 32  # 初始通道数64，增长率32（每个卷积块新增32个通道）
num_convs_in_dense_blocks = [4, 4, 4, 4]  # 定义4个稠密块，每个块包含4个卷积层
blks = []  # 创建空列表，用于存储所有的稠密块和过渡层

for i, num_convs in enumerate(num_convs_in_dense_blocks):  # 遍历每个稠密块
    # 添加一个稠密块
    # num_convs: 当前块中的卷积层数量
    # num_channels: 当前块的输入通道数
    # growth_rate: 每层新增的通道数
    blks.append(DenseBlock(num_convs, num_channels, growth_rate))
    
    # 计算稠密块的输出通道数
    # 输出通道数 = 输入通道数 + 卷积层数 × 增长率
    num_channels += num_convs * growth_rate
    
    # 在稠密块之间添加过渡层（最后一个稠密块后不需要过渡层）
    if i != len(num_convs_in_dense_blocks) - 1:
        # 过渡层将通道数减半，控制模型复杂度
        blks.append(transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2  # 更新通道数为减半后的值

In [ ]:
# 构建完整的DenseNet网络
net = nn.Sequential(
    b1,  # 第一个模块：包含大卷积和最大池化
    *blks,  # 使用*解包，将列表中所有的稠密块和过渡层依次添加
    nn.BatchNorm2d(num_channels),  # 最后一个稠密块后的批量归一化
    nn.ReLU(),  # ReLU激活函数
    nn.AdaptiveAvgPool2d((1, 1)),  # 自适应平均池化，将任意大小的特征图变为1x1
    nn.Flatten(),  # 展平层，将多维张量压缩为一维向量
    nn.Linear(num_channels, 10))  # 全连接层，输出10个类别的分数（对应Fashion-MNIST的10个类别）

In [ ]:
# 设置训练参数并开始训练
lr, num_epochs, batch_size = 0.1, 10, 256  # 学习率0.1，训练10个epoch，批量大小256
# 加载Fashion-MNIST数据集，resize=96将图像调整为96x96
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
# 使用d2l提供的训练函数训练模型
# net: 网络模型
# train_iter: 训练数据迭代器
# test_iter: 测试数据迭代器
# num_epochs: 训练轮数
# lr: 学习率
# d2l.try_gpu(): 如果有GPU则使用GPU，否则使用CPU
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())